In [43]:
import csv
import codecs
import urllib.request
import sys
from urllib.parse import urlparse
import json

In [2]:
# This is the core of our weather query URL
BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/'

In [69]:
# Set up the location parameter for our query
QueryLocation = '&location=43.232212,-70.829685' 

# Set up the query type parameter for our query ('FORECAST' or 'HISTORY')
QueryType= 'HISTORY'

# Set up the key parameter for our query
QueryKey = '&key=3IG6UAJ4EUV8THM9KCEX13TMU'

# Set up the date parameters for our query. Used only for historical weather data requests
FromDateParam = '2021-1-6'
ToDateParam = '2021-3-8'

#Included sections
Include = '&include=obs%2Cfcst%2Cstats%2Calerts%2Ccurrent%2Cevents%2Chistfcst'

content_type = '&contentType=csv'

In [70]:
# Set up the specific parameters based on the type of query
if QueryType == 'FORECAST':
    print(' - Fetching forecast data')
    QueryTypeParams = 'forecast?&aggregateHours=24&unitGroup=us&shortColumnNames=false'
else:
    print(' - Fetching history for date: ', FromDateParam,'-',ToDateParam)

    # History requests require a date.  We use the same date for start and end since we only want to query a single date in this example
    QueryDate = '&startDateTime=' + FromDateParam + 'T00:00:00&endDateTime=' +ToDateParam + 'T00:00:00'
    QueryTypeParams = 'history?&aggregateHours=24&unitGroup=us&dayStartTime=0:0:00&dayEndTime=0:0:00' + QueryDate

 - Fetching history for date:  2021-1-6 - 2021-3-8


In [71]:
# Build the entire query
URL = BaseURL + QueryTypeParams + content_type + QueryLocation + QueryKey + Include

print(' - Running query URL: ', URL)
print()

 - Running query URL:  https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=24&unitGroup=us&dayStartTime=0:0:00&dayEndTime=0:0:00&startDateTime=2021-1-6T00:00:00&endDateTime=2021-3-8T00:00:00&contentType=csv&location=43.232212,-70.829685&key=3IG6UAJ4EUV8THM9KCEX13TMU&include=obs%2Cfcst%2Cstats%2Calerts%2Ccurrent%2Cevents%2Chistfcst



In [72]:
# Parse the results as CSV
CSVBytes = urllib.request.urlopen(URL)
CSVText = csv.reader(codecs.iterdecode(CSVBytes, 'utf-8'))

In [74]:
RowIndex = 0

# The first row contain the headers and the additional rows each contain the weather metrics for a single day
# To simply our code, we use the knowledge that column 0 contains the location and column 1 contains the date.  The data starts at column 4
for Row in CSVText:
    if RowIndex == 0:
        FirstRow = Row
    else:
        print('Weather in ', Row[0], ' on ', Row[1])

        ColIndex = 0
        for Col in Row:
            if ColIndex >= 4:
                print('   ', FirstRow[ColIndex], ' = ', Row[ColIndex])
            ColIndex += 1
    RowIndex += 1

Weather in  43.232212,-70.829685  on  01/06/2021
    Temperature  =  32.4
    Dew Point  =  21.9
    Relative Humidity  =  65.29
    Heat Index  =  
    Wind Speed  =  9.2
    Wind Gust  =  17.2
    Wind Direction  =  304.67
    Wind Chill  =  23.6
    Precipitation  =  0.0
    Precipitation Cover  =  0.0
    Snow Depth  =  0.39
    Visibility  =  9.9
    Cloud Cover  =  48.5
    Sea Level Pressure  =  1011.2
    Weather Type  =  Snow Showers
    Latitude  =  43.232212
    Longitude  =  -70.829685
    Resolved Address  =  43.232212,-70.829685
    Name  =  43.232212,-70.829685
    Info  =  
    Conditions  =  Partially cloudy
Weather in  43.232212,-70.829685  on  01/07/2021
    Temperature  =  31.4
    Dew Point  =  17.1
    Relative Humidity  =  55.4
    Heat Index  =  
    Wind Speed  =  8.8
    Wind Gust  =  18.3
    Wind Direction  =  300.88
    Wind Chill  =  18.5
    Precipitation  =  0.0
    Precipitation Cover  =  0.0
    Snow Depth  =  0.13
    Visibility  =  9.9
    Cloud Cove

    Wind Gust  =  20.8
    Wind Direction  =  298.46
    Wind Chill  =  -2.1
    Precipitation  =  0.0
    Precipitation Cover  =  0.0
    Snow Depth  =  9.42
    Visibility  =  9.9
    Cloud Cover  =  26.2
    Sea Level Pressure  =  1024.7
    Weather Type  =  
    Latitude  =  43.232212
    Longitude  =  -70.829685
    Resolved Address  =  43.232212,-70.829685
    Name  =  43.232212,-70.829685
    Info  =  
    Conditions  =  Partially cloudy
Weather in  43.232212,-70.829685  on  02/13/2021
    Temperature  =  21.5
    Dew Point  =  0.2
    Relative Humidity  =  39.41
    Heat Index  =  
    Wind Speed  =  4.3
    Wind Gust  =  
    Wind Direction  =  161.79
    Wind Chill  =  6.4
    Precipitation  =  0.0
    Precipitation Cover  =  0.0
    Snow Depth  =  9.33
    Visibility  =  9.9
    Cloud Cover  =  30.4
    Sea Level Pressure  =  1026.1
    Weather Type  =  
    Latitude  =  43.232212
    Longitude  =  -70.829685
    Resolved Address  =  43.232212,-70.829685
    Name  =  43.2322

In [83]:
# If there are no CSV rows then something fundamental went wrong
if RowIndex == 0:
    print('Sorry, but it appears that there was an error connecting to the weather server.')
    print('Please check your network connection and try again..')

# If there is only one CSV  row then we likely got an error from the server
if RowIndex == 1:
    print('Sorry, but it appears that there was an error retrieving the weather data.')
    print('Error: ', FirstRow)

In [18]:
weather_cols = ['Address','Date time','Minimum Temperature','Maximum Temperature','Temperature','Dew Point',
                'Relative Humidity','Heat Index','Wind Speed','Wind Gust','Wind Direction','Wind Chill',
                'Precipitation','Precipitation Cover','Snow Depth','Visibility','Cloud Cover','Sea Level Pressure',
                'Weather Type','Latitude','Longitude','Resolved Address','Name','Info','Conditions']